Evaluating the error from sharpening GOES images from bands 7 and 14 using laplace and unsharpmask sharpening.

In [1]:
import os
import random

import numpy as np
import pandas as pd

from skimage.filters import laplace
from skimage.filters import unsharp_mask
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    prefix = file[:19] + band + file[21:41]
    for x in os.listdir(path):
        if x.startswith(prefix):
            return x

In [3]:
def control(path, file):
    """
    Returns a dictionary of errors for the given file without sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to evaluate the error of
    Precondition: file is a string
    """
    img = np.load(path + file)
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    mae = mean_absolute_error(img, control)
    rmse = mean_squared_error(img, control, squared=False)
    return {'File': file, 'Band': file[19:21], 'Control_MAE': mae, 'Control_RMSE': rmse}

def laplace_sharpening(path, file):
    """
    Returns a dictionary of errors for the given file using laplace sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    img = np.load(path + file)
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + laplace_edges
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'File': file, 'Band': file[19:21], 'LP_MAE': mae, 'LP_RMSE': rmse}

def unsharpmask_sharpening(path, file):
    """
    Returns a dictionary of errors for the given file using unsharpmask sharpening. To normalize images,
    each image is divided by its maximum value, and then multiplied by the same value after sharpening
    is completed.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    img = np.load(path + file)
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'File': file, 'Band': file[19:21], 'UM_MAE': mae, 'UM_RMSE': rmse}

In [4]:
path = '../../GOES_Files/npy_files/'

file07 = ''
while file07[19:21] != '07':
    file07 = random.choice(os.listdir(path))
file14 = find_npy_file(path, file07, '14')

data07 = {}
data07.update(control(path, file07))
data07.update(laplace_sharpening(path, file07))
data07.update(unsharpmask_sharpening(path, file07))

data14 = {}
data14.update(control(path, file14))
data14.update(laplace_sharpening(path, file14))
data14.update(unsharpmask_sharpening(path, file14))

data = []
data.append(data07)
data.append(data14)
data = pd.DataFrame(data)

In [5]:
data

,File,Band,Control_MAE,Control_RMSE,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE
0,OR_ABI-L1b-RadC-M3C07_G17_s20183291147190_e201...,07,0.517006,0.731280,0.420362,0.603949,0.432304,0.615138
1,OR_ABI-L1b-RadC-M3C14_G17_s20183291147190_e201...,14,0.547612,0.819331,0.443080,0.673185,0.455920,0.687484
